In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [10]:
train_nlu_model_switch = False

In [4]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   10.0s finished


In [5]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [8]:
query = 'do i need a visa to visit india'

In [9]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.8625167454063546,
   'end': 16,
   'entity': 'requirement',
   'extractor': 'ner_crf',
   'start': 12,
   'value': 'visa'}],
 'intent': {'confidence': 0.72160047624340007, 'name': 'requirement'},
 'intent_ranking': [{'confidence': 0.72160047624340007, 'name': 'requirement'},
  {'confidence': 0.052034770670308317, 'name': 'procedure'},
  {'confidence': 0.037175232513816221, 'name': 'validity'},
  {'confidence': 0.029378835999498559, 'name': 'definition'},
  {'confidence': 0.024564056383751899, 'name': 'purpose'},
  {'confidence': 0.023878036819287678, 'name': 'duration'},
  {'confidence': 0.016219502794496229, 'name': 'email'},
  {'confidence': 0.014650138932202841, 'name': 'conditions'},
  {'confidence': 0.014244593386735361, 'name': 'extent'},
  {'confidence': 0.013050230800543737, 'name': 'address'}],
 'text': 'do i need a visa to visit india'}